In [1]:
import requests
from bs4 import BeautifulSoup
import re
import pandas as pd
from matplotlib import pyplot as plt
import seaborn as sns
import time
import pickle

In [4]:
df = pickle.load( open( "criterion.pkl", "rb" ) )

In [5]:
df

,title,director,country,year,links,summaries,summary
0,2 or 3 Things I Know About Her,Jean-Luc Godard,"France,",1967,https://www.criterionchannel.com/2-or-3-things...,Directed by Jean-Luc Godard • 1967 • France\nS...,Directed by Jean-Luc Godard • 1967 • France St...
1,3:10 to Yuma,Delmer Daves,"United States,",1957,https://www.criterionchannel.com/3-10-to-yuma,Directed by Delmer Daves • 1957 • United State...,Directed by Delmer Daves • 1957 • United State...
2,3 Faces,Jafar Panahi,"Iran,",2018,https://www.criterionchannel.com/3-faces,Directed by Jafar Panahi • 2018 • Iran\nStarri...,Directed by Jafar Panahi • 2018 • Iran Starrin...
3,"4 Months, 3 Weeks and 2 Days",Cristian Mungiu,"Romania,",2007,https://www.criterionchannel.com/4-months-3-we...,Criterion Collection Edition #958\r\n\r\nRoman...,Criterion Collection Edition #958 Romanian ...
4,5 Against the House,Phil Karlson,"United States,",1955,https://www.criterionchannel.com/5-against-the...,Directed by Phil Karlson • 1955 • United State...,Directed by Phil Karlson • 1955 • United State...
...,...,...,...,...,...,...,...
2095,Zéro de conduite,Jean Vigo,"France,",1933,https://www.criterionchannel.com/zero-de-conduite,Directed by Jean Vigo • 1933 • France\nStarrin...,Directed by Jean Vigo • 1933 • France Starring...
2096,Zero Focus,Yoshitaro Nomura,"Japan,",1961,https://www.criterionchannel.com/zero-focus,Directed by Yoshitaro Nomura • 1961 • Japan\n\...,Directed by Yoshitaro Nomura • 1961 • Japan A...
2097,Zora Neale Hurston Fieldwork Footage (excerpt),Zora Neale Hurston,"United States,",1928,https://www.criterionchannel.com/zora-neale-hu...,Directed by Zora Neale Hurston • 1928 • United...,Directed by Zora Neale Hurston • 1928 • United...
2098,Zorgon: The H-Bomb Beast from Hell,Kevin Fernan,"United States,",1972,https://www.criterionchannel.com/zorgon-the-h-...,Directed by Kevin Fernan • 1972 • United State...,Directed by Kevin Fernan • 1972 • United State...
